In [1]:
import pandas as pd
import re
import evaluate
import numpy as np
from transformers import BertTokenizer, BertForSequenceClassification, TrainingArguments, Trainer
import torch
from torch.optim import AdamW, lr_scheduler
import torch.nn as nn
import matplotlib.pyplot as plt
import numpy as np
from scipy.interpolate import make_interp_spline, BSpline
import nltk
from sklearn.model_selection import train_test_split
from torch.utils.data import Dataset


In [2]:
cases = pd.read_csv("justice.csv")
#print(cases.isna().sum())
cases.dropna(inplace=True)
#print(cases.isna().sum())
#cases.head()

In [3]:
max_idx = cases['facts_len'].idxmax()

# extract the text from the 'facts' column for the selected row
longest_facts = cases.loc[max_idx, 'facts']

#print(longest_facts)
cases.drop(columns=['Unnamed: 0', 'ID', 'name', 'href', 'docket', 'term',  
                    'majority_vote', 'minority_vote', 'decision_type', 'disposition', 'issue_area'], inplace=True)



In [4]:
# Get the fact lengths
fact_lengths = cases['facts_len'].tolist()

# Assuming fact_lengths is a NumPy array or list of fact lengths
unique_lengths = np.unique(fact_lengths)

# Create a sorted array of unique fact lengths and their corresponding indices
sorted_lengths = np.sort(unique_lengths)
indices = np.arange(len(sorted_lengths))

# Interpolate the indices using a cubic spline
interp_func = make_interp_spline(sorted_lengths, indices, k=3)

# Generate a smooth curve by evaluating the interpolation function over a range of fact lengths
x = np.linspace(min(fact_lengths), max(fact_lengths), num=1000)
y = interp_func(x)

# Plot the curve using Matplotlib
import matplotlib.pyplot as plt
#plt.plot(x, y)
#plt.xlabel('Fact Length')
#plt.ylabel('Cumulative Density')
#plt.show()

In [5]:
# Create a scatterplot of the fact lengths
#plt.scatter(range(len(fact_lengths)), fact_lengths, s=5, alpha=0.5)

# Add labels and title
#plt.xlabel('Index')
#plt.ylabel('Fact Length')
#plt.title('Distribution of Fact Lengths')

# Show the plot
#plt.show()

# Count the number of facts below 2500 length
num_short_facts = len(cases[cases['facts_len'] < 2500])

# Calculate the percentage of short facts
percentage_short_facts = num_short_facts / len(cases) * 100
#print(percentage_short_facts)

In [6]:
cases = cases.rename(columns={'first_party_winner': 'winning_party_idx'})
for i, row in cases.iterrows():
    if row['winning_party_idx'] == True:
        cases.loc[i, 'winning_party_idx'] = 0
    else:
        cases.loc[i, 'winning_party_idx'] = 1
#cases

In [7]:
cases['facts'] = cases['facts'].str.replace(r'<[^<]+?>', '', regex=True)
cases['facts'] = cases['facts'].str.lower().apply(lambda x: re.sub(r'[^a-z\s]', '', x)).apply(lambda x: ' '.join([word for word in x.split() if len(word) > 1]))
#cases

use bert for classification
use sep token to include parties in case facts

tokenizer.sep_token

df.first_party + " [SEP] " + df.second_party + " [sep] " + 

bert for sequence classification 

gpt-2 simple 

In [8]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
cases['token_facts'] = cases.apply(lambda x: f"{x['first_party']} [SEP] {x['second_party']} [SEP] {x['facts']}", axis=1)
cases = cases.drop(columns=['facts', 'first_party', 'second_party', 'facts_len'])
cases['token_facts'] = cases['token_facts'].apply((lambda row: tokenizer.encode(row, add_special_tokens=True, padding="max_length",
                                                                         max_length=256, truncation=True)))
train_cases, test_cases = train_test_split(
    cases, test_size=0.25)

class TextDataset(Dataset): 
    def __init__(self, text: list, labels: list):
        self.text = torch.Tensor(text)
        self.labels = torch.Tensor(labels)
    
    def __getitem__(self, idx: int) -> tuple[str, int]:
        return self.text[idx], self.labels[idx]
    
    def __len__(self) -> int:
        return self.labels.size()

train_dataset = TextDataset(train_cases['token_facts'].tolist(), train_cases['winning_party_idx'].tolist())
test_dataset = TextDataset(test_cases['token_facts'].tolist(), test_cases['winning_party_idx'].tolist())

    

In [9]:
#Load pretrained model
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=2)
training_args = TrainingArguments(output_dir="test_trainer", evaluation_strategy="epoch")
metric = evaluate.load("accuracy")
optimizers = tuple([AdamW(model.parameters()), lr_scheduler.LambdaLR])
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=1)
    return metric.compute(predictions=predictions, references=labels)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_cases,
    eval_dataset=test_cases,
    compute_metrics=compute_metrics,
    optimizers=optimizers
)

trainer.train()

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

KeyError: 751